In [1]:
%run Functions.ipynb

In [2]:
import tweepy 
import json
import datetime
import sqlite3
from sqlite3 import Error
import numpy as np

In [3]:
def create_connection(db_file):
    connection = None
    try:
        connection = sqlite3.connect(db_file)
        return connection
    except Error as e:
        print(e)
 
    return connection

In [4]:
def create_all_tables():
    #Sentiment Table
    sql_create_sentiments_table = """ CREATE TABLE IF NOT EXISTS sentiment_table (
                                    sentiment_id text PRIMARY KEY,
                                    user_name text NOT NULL,
                                    tweet text,
                                    longitude float,
                                    latitude float,
                                    sentiment text
                                ); """
    create_table(sql_create_sentiments_table)
    
    #Graph Table
    sql_create_graph_table = """ CREATE TABLE IF NOT EXISTS graph_table (
                                    graph_id text PRIMARY KEY,
                                    id text,
                                    text text,
                                    screen_name text,
                                    user_id text,
                                    followers_count text,
                                    user_mentions_screen_name text,
                                    user_mentions_id text,
                                    retweeted_screen_name text,
                                    retweeted_id text, 
                                    in_reply_to_screen_name text,
                                    in_reply_to_status_id text,
                                    in_reply_to_user_id text
                                ); """
    create_table(sql_create_graph_table)

    #word frequency count table
    sql_create_word_table = """CREATE TABLE IF NOT EXISTS word_table (
                                    word_id text PRIMARY KEY,
                                    id text,
                                    text text,
                                    screen_name text,
                                    user_id text
                                ); """
    create_table(sql_create_word_table)
    
   
    #hashtag table 
    sql_create_hashtag_table = """CREATE TABLE IF NOT EXISTS hashtag_table (
                                 hashtag_id text PRIMARY KEY,
                                 id text,
                                 text text,
                                 screen_name text, 
                                 user_id text,
                                 hashtags text,
                                 polls text,
                                 source text,
                                 reply_count text,
                                 retweet_count text
                                 
                            ); """
    create_table(sql_create_hashtag_table)

In [5]:
def create_table(create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
        conn.commit()
        c.close()   
    except Error as e:
        print(e)

In [6]:
def insert_into_table(insert_sql, parameters):
    try:
        c = conn.cursor()
        c.execute(insert_sql, parameters)
        conn.commit()
        c.close()   
    except Error as e:
        print(e)

In [7]:
def insert_into_sentiment_table(data):
        
    if data['coordinates'] is not None:
        sentiment_id = term_to_search + "_"+ str(datetime.datetime.now().timestamp())
        user_name = data['user']['name']
        tweet = data['text']
        longitude = data['coordinates'][0]
        latitude = data['coordinates'][1]
        sentiment = ""
    elif data['place'] is not None:
        bbox = data['place']['bounding_box']['coordinates']
        sentiment_id = term_to_search + "_"+ str(datetime.datetime.now().timestamp())
        user_name = data['user']['name']
        tweet = data['text']
        longitude = bbox[0][0][0]
        latitude = bbox[0][0][1]
        sentiment = ""
    else:
        sentiment_id = None
    
    if sentiment_id is not None:
        
        insert_sql = '''INSERT INTO sentiment_table VALUES (?, ?, ?, ?, ?, ?)'''
        parameters = (sentiment_id, user_name, tweet, longitude, latitude, sentiment)
               
        insert_into_table(insert_sql, parameters)   
        
    return

In [8]:
def insert_into_graph_table(data):
        
    graph_id = term_to_search + "_graphtable_"+ str(datetime.datetime.now().timestamp())  
    g_id = data['id']
    text = data['text']
    
    screen_name = data['user']['screen_name']
    user_id = data['user']['id']
    followers_count = data['user']['followers_count']
    user_mentions = data['entities']['user_mentions']
    
    if user_mentions is not None:
        user_mentions_screen_name = user_mentions[0]['screen_name']
        user_mentions_id = user_mentions[0]['id_str']
    else:
        user_mentions_screen_name = None
        user_mentions_id = None
        
    retweeted_status = data['retweeted_status']
    
    if retweeted_status is not None:
        retweeted_screen_name = retweeted_status['user']['screen_name']
        retweeted_id = retweeted_status['user']['id_str']
    else:
        retweeted_screen_name = None
        retweeted_id = None
                                     
    in_reply_to_screen_name = data['in_reply_to_screen_name']
    in_reply_to_status_id = data['in_reply_to_status_id']
    in_reply_to_user_id = data['in_reply_to_user_id']
      
    insert_sql = '''INSERT INTO graph_table VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'''
    parameters = (graph_id, g_id, text, screen_name, user_id, followers_count,
                  user_mentions_screen_name, user_mentions_id, retweeted_screen_name, retweeted_id,
                  in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id
                 )

    insert_into_table(insert_sql, parameters)   
    
    return

In [9]:
def insert_into_word_table(data):
    
    word_id = term_to_search + "_wordtable_"+ str(datetime.datetime.now().timestamp())  
    w_id = data['id']
    text = data['text']
    screen_name = data['user']['screen_name']
    user_id = data['user']['id']
    
    insert_sql = '''INSERT INTO word_table VALUES (?, ?, ?, ?, ?)'''
    parameters = (word_id, w_id, text, screen_name, user_id)

    insert_into_table(insert_sql, parameters)   
    
    return

In [10]:
def insert_into_hashtag_table(data):
    
    hashtag_id = term_to_search + "_hashtagtable_" + str(datetime.datetime.now().timestamp())
    h_id = data['id']
    text = data['text']
    screen_name = data['user']['screen_name']
    user_id = data['user']['id']  

    hashtags = data['entities']['hashtags'][0]['text']
    
   

    polls = None

#     data['polls']
    source = data['source']
    reply_count = data['reply_count']
    retweet_count = data['retweet_count']
    
    insert_sql = '''INSERT INTO hashtag_table VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'''
    parameters = (hashtag_id, h_id, text, screen_name, user_id, hashtags, polls, source, reply_count, retweet_count)
    
    insert_into_table(insert_sql, parameters)
    
    return

In [11]:
class streamListener(tweepy.StreamListener):

    
    def on_data(self, raw_data):
       
        
        try:
                       
            data = json.loads(raw_data) 
            
            insert_into_sentiment_table(data)
#             insert_into_graph_table(data)
#             insert_into_word_table(data)
            insert_into_hashtag_table(data)
    
        except Exception as e:
            print(e)
    
    def on_error(self, status_code):
        if status_code == 420:
            return False

In [ ]:
if __name__ == "__main__": 

    global term_to_search, conn
    
    database = "Twitter_Database.db"   
    conn = create_connection(database)
    
    create_all_tables()          
            
    term_to_search = "elections"
    
    api = tweepy.API(wait_on_rate_limit_notify=True)

    streamer = tweepy.Stream(auth=autorize_twitter_api(), listener=streamListener(api))
    streamer.filter(languages=["en"], track=[term_to_search])

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o